In [9]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras import backend as K
from os import listdir
from os.path import isfile, join
from sklearn.utils import shuffle
import cv2
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import load_model
import pickle

batch_size = 100
epochs = 4
INIT_LR = 1e-3

# input image dimensions
img_rows, img_cols = 25, 44

rpath = "F:/Google Drive/User_Backup/training"
h_max, w_max, h_out, w_out = 75, 132, 25, 44
image = []
labels=[]
mlb =[]
for j in range (0, 101):
    print(j)
    for k in range (5000):
        tpath = rpath + "/" + str(j) + "/" + str(k) + ".png"
        img = cv2.imread(tpath, 0)
        if img.size ==0:
            print(tpath)
        image.append(img)
        if j == 100:
            labels.append((" "," "))
        else:
            lo = str(j%10)
            t = j//10
            if t == 0:
                lt = "t "
            else:
                lt = "t" + str(t)
            labels.append((lt,lo))
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
#print(labels[0])
print(len(image))
print(len(labels))
train_image, labels = shuffle(image, labels)
i = 0

x_train = np.array(train_image[:49999])
x_test = np.array(train_image[50000:50499])
y_train = np.array(labels[:49999])
y_test = np.array(labels[50000:50499])
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
#y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, len(mlb.classes_))
#y_test = keras.utils.to_categorical(y_test, len(mlb.classes_))

# initialize the model along with the input shape to be
# "channels last" and the channels dimension itself
model = Sequential()
chanDim = -1
            
# CONV => RELU => POOL
model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
        
# (CONV => RELU) * 2 => POOL
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
       
# (CONV => RELU) * 2 => POOL
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
       
# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# softmax classifier
model.add(Dense(21))
model.add(Activation("sigmoid"))

#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])
opt = Adam(lr=INIT_LR, decay=INIT_LR / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("F:/Google Drive/User_Backup/model/multi_labels.h5")

f = open("F:/Google Drive/User_Backup/model/labelbin", "wb")
f.write(pickle.dumps(mlb))
f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
505000
505000
x_train shape: (49999, 25, 44, 1)
49999 train samples
499 test samples
Train on 49999 samples, validate on 499 samples
Epoch 1/4
49999/49999 [==============================] - 87s 2ms/sample - loss: 0.2496 - accuracy: 0.8929 - val_loss: 0.1839 - val_accuracy: 0.9208
Epoch 2/4
49999/49999 [==============================] - 86s 2ms/sample - loss: 0.0603 - accuracy: 0.9792 - val_loss: 0.0381 - val_accuracy: 0.9863
Epoch 3/4
49999/49999 [==============================] - 85s 2ms/sample - loss: 0.0385 - accuracy: 0.9871 - val_loss: 0.0196 - val_accuracy: 0.9924
Epoch 4/4
49999/49999 [==============================] - 85s 2ms/sample - loss: 0.0296 - accuracy: 0.9902 - val_loss: 0.0178 - va

In [6]:
print(y_test[0])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0.

In [5]:
print(labels.shape)

(50500, 21)


In [12]:
from keras.models import load_model
model.save(args["multi_labels_model"], save_format="h5")

NameError: name 'args' is not defined

In [15]:
import pickle
f = open("F:/Google Drive/User_Backup/model/labelbin", "wb")
f.write(pickle.dumps(mlb))
f.close()

In [16]:
model.save("F:/Google Drive/User_Backup/model/multi_labels.h5")